In [76]:
import boto3
import base64
import os
import json 
# from object_store import ObjectStore
import yaml

s3_configure_path = './config/s3-service-key-cesco1-interim.txt'

with open(os.path.join(os.getcwd(), s3_configure_path)) as f:
    os_env_c = json.load(f)
    aws_access_key_id = os_env_c['access_key_id']
    aws_secret_access_key = os_env_c['secret_access_key']
    aws_region = os_env_c['region']
    
    ### Update path 
    aws_url = os.path.join('s3://', os_env_c['bucket'])
    bucket_name = os_env_c['bucket']


# Initialize the S3 client
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)

pdf_file_key = 'test_cesco/test3.pdf'#store.get('test3.pdf') #'path/to/your-pdf-file.pdf'

print(pdf_file_key)
# Download the PDF file from the Object Store
response = s3_client.get_object(Bucket=bucket_name, Key=pdf_file_key)
pdf_data = response['Body'].read()

print(bucket_name)
print(pdf_file_key)
# # Optionally, convert the PDF to base64 (for embedding in YAML or other files)
# base64_pdf = base64.b64encode(pdf_data).decode('utf-8')

# print(base64_pdf)

test_cesco/test3.pdf
hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72
test_cesco/test3.pdf


In [77]:
import boto3

s3 = boto3.client('s3')

# List all objects in the bucket
response = s3_client.list_objects_v2(Bucket=bucket_name)

# for obj in response.get('Contents', []):
#     print(obj['Key'])
#     head_response = s3_client.head_object(Bucket=bucket_name, Key=pdf_file_key)
#     print(f"Key: {pdf_file_key}, Content-Type: {head_response['ContentType']}")



In [78]:
print(bucket_name)

hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72


In [86]:
import boto3
from botocore.exceptions import ClientError
from botocore.client import Config


s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region,
    config=Config(signature_version='s3v4')
)

# 버킷하고 키 설정
bucket_name = 'hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72'
source_key = 'test_cesco/test3.pdf'  # 원본 객체 키
destination_key = 'test_cesco/test3_metacopy.pdf'  # 복사 후 객체의 키

try:
    # 원본 객체 메타데이터 가져오기
    head_response = s3_client.head_object(Bucket=bucket_name, Key=source_key)
    
    # 기존 메타데이터 복사
    metadata = head_response['Metadata']
    content_type = head_response.get('ContentType', 'binary/octet-stream')  # 기존 Content-Type 유지
    
    # 새로운 Content-Type 및 Content-Disposition 설정
    metadata.update({'Content-Type': 'application/pdf'})  # PDF로 강제 설정
    
    # 원본 객체 복사 및 메타데이터 수정
    s3_client.copy_object(
        Bucket=bucket_name,
        CopySource={'Bucket': bucket_name, 'Key': source_key},
        Key=destination_key,
        MetadataDirective='REPLACE',  # 메타데이터 덮어쓰기
        Metadata=metadata,  # 수정된 메타데이터 적용
        ContentType='application/pdf',  # 브라우저에서 열리게 하기 위한 타입 변경
        ContentDisposition='inline',  # 브라우저에서 열리게 하기 위한 타입 변경
        ACL='public-read'  # 퍼블릭 액세스 설정
    )
    
    print(f"객체 {source_key}가 {destination_key}로 성공적으로 복제되었습니다.")

except ClientError as e:
    print(f"Error: {e}")

NameError: name 'Config' is not defined

In [84]:
response_head_updated = s3_client.head_object(Bucket='hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72', Key='test_cesco/test3_metacopy1.pdf')

# 메타데이터 출력
print("Content-Type:", response_head_updated.get('ContentType', 'Not Set'))
print("Content-Disposition:", response_head_updated.get('ContentDisposition', 'Not Set'))

Content-Type: binary/octet-stream
Content-Disposition: Not Set


In [60]:
import boto3
import base64
import os
import json 
# from object_store import ObjectStore
import yaml

s3_configure_path = './config/s3-service-key-cesco1-interim.txt'

with open(os.path.join(os.getcwd(), s3_configure_path)) as f:
    os_env_c = json.load(f)
    aws_access_key_id = os_env_c['access_key_id']
    aws_secret_access_key = os_env_c['secret_access_key']
    aws_region = os_env_c['region']
    
    ### Update path 
    aws_url = os.path.join('s3://', os_env_c['bucket'])
    bucket_name = os_env_c['bucket']


# Initialize the S3 client
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)

# s3_client = boto3.client('s3')

# 기존 객체의 메타데이터 조회
response_head = s3_client.head_object(Bucket='hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72', Key='test_cesco/test2.pdf')

# 기존 메타데이터 가져오기
existing_metadata = response_head.get('Metadata', {})

# 메타데이터 복사 및 추가할 값 설정
new_metadata = existing_metadata.copy()
new_metadata.update({
    'Content-Type': 'application/pdf',
    'Content-Disposition': 'inline'  # 브라우저에서 PDF 파일을 바로 열도록 설정
})

# 객체를 복사하면서 메타데이터 덮어쓰기
response_copy = s3_client.copy_object(
    Bucket='hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72',
    CopySource={'Bucket': 'hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72', 'Key': 'test_cesco/test2.pdf'},
    Key='test_cesco/test2.pdf',
    MetadataDirective='REPLACE',  # 메타데이터 교체
    Metadata=new_metadata  # 새 메타데이터
)

response_acl = s3_client.put_object_acl(
    Bucket='hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72',
    Key='test_cesco/test2.pdf',
    ACL='public-read'  # 퍼블릭 액세스 설정
)

# 복사 후 메타데이터 확인
response_head_updated = s3_client.head_object(Bucket='hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72', Key='test_cesco/test2.pdf')

# 메타데이터 출력
print("Content-Type:", response_head_updated.get('ContentType', 'Not Set'))
print("Content-Disposition:", response_head_updated.get('ContentDisposition', 'Not Set'))

ClientError: An error occurred (SignatureDoesNotMatch) when calling the CopyObject operation: The request signature we calculated does not match the signature you provided. Check your key and signing method.

In [82]:
import boto3

# s3 = boto3.client('s3')

response_copy = s3_client.copy_object(
    Bucket='hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72',
    CopySource={'Bucket': 'hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72', 'Key': 'test_cesco/test2.pdf'},
    Key='test_cesco/test2.pdf',
    MetadataDirective='REPLACE',  # 메타데이터 대체
    Metadata={
        'Content-Type': 'application/pdf',
        'Content-Disposition': 'inline'  # PDF를 브라우저에서 직접 보기 위해 설정
    }
)

response_acl = s3_client.put_object_acl(
    Bucket='hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72',
    Key='test_cesco/test2.pdf',
    ACL='public-read'  # 퍼블릭 액세스 설정
)

# S3 객체의 메타데이터 확인
response_head = s3_client.head_object(Bucket='hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72', Key='test_cesco/test3_metacopy.pdf')

# 메타데이터 출력
print("Content-Type:", response_head.get('ContentType', 'Not Set'))
print("Content-Disposition:", response_head.get('ContentDisposition', 'Not Set'))


# https://hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72.s3-ap-northeast-1.amazonaws.com/test_cesco/test2.pdf

# print(response_acl)


# response = s3_client.put_object_acl(
#     Bucket='hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72',
#     Key='cesco_division_file/cesco2_0.pdf',
#     MetadataDirective='REPLACE',  # 기존 메타데이터를 대체
#     ContentType='application/pdf',
#     ACL='public-read'
# )

Content-Type: binary/octet-stream
Content-Disposition: Not Set


In [ ]:
response = s3_client.head_object(Bucket='hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72', Key='test_cesco/test3.pdf')

# 메타데이터 출력
print("Content-Type:", response.get('ContentType', 'Not Set'))
print("Content-Disposition:", response.get('ContentDisposition', 'Not Set'))

In [ ]:
# https://hcp-a8f3d857-56e7-4be2-a9d1-d558ccd5da72.s3-ap-northeast-1.amazonaws.com/test_cesco/test3.pdf

# https://<bucket-name>.s3.<region>.amazonaws.com/<object-key>
# https://my-bucket.s3.eu-west-1.amazonaws.com/my-folder/my-file.pdf
